In [1]:
# Code to work in Google Colab
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Leave this
num_epochs = 10

# TUNE THESE to achieve maximum performance
batch_size = 16
learning_rate = 3e-2

In [3]:
import os

GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', 'EECS 545 Project')
print(os.listdir(GOOGLE_DRIVE_PATH))

os.chdir(GOOGLE_DRIVE_PATH)
print(os.getcwd())

['metadata.json', 'Image.zip', '545 Project Ideas.gdoc', 'data_scene_flow', 'Copy of Data_Loading.ipynb', '.ipynb_checkpoints', 'Image.zip (Unzipped Files)', 'Images', 'Untitled Diagram.drawio', 'Weather', 'Extra_Weather_Data', 'More_Weather_Data_60k', 'DataSplit.ipynb', 'Unet + triplet loss.ipynb', 'Data_Loading.ipynb', 'saved_model', 'Untitled', '__pycache__', 'Joey Unet and Content Loss.ipynb', 'Test version(Yuanbin)-UNet Triplet loss.ipynb', 'JoeyVAE598.py', 'ExampleInterpolation.png', 'JoeyTransferLearning.ipynb', 'newest Unet and Triplet Loss.ipynb', 'WeatherVAE.ipynb', 'WeatherVAE.pt', 'Tune parameters.ipynb', 'Baselines.ipynb']
/content/drive/.shortcut-targets-by-id/1YO9ukO9embmobkuEcbxl71Qpmtma6P0Q/EECS 545 Project


In [4]:
import pandas as pd
import torch.nn as nn
from PIL import Image
from torchvision import transforms

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [5]:
class SmallDataLoader:
  def __init__(self, BatchSize=16, TestPercent=0.1, data_loc="Weather", img_size=224, dev="cuda"):
    self.train_names = []
    self.train_labels = []
    self.val_names = []
    self.val_labels = []
    self.test_names = []
    self.test_labels = []

    self.label_keys = {0: "cloudy", 1: "foggy", 2: "rain", 3: "snow", 4: "sunny"}
    self.data_loc = os.path.join(os.getcwd(), data_loc)
    self.batch_size = BatchSize
    self.img_size = img_size
    self.dev=dev

    data_types = ["cloudy", "foggy", "rain", "snow", "sunny"]
    splits = ["Train", "Val", "Test"]
    for split in splits:
      for i in range(5):
        dtype = self.label_keys[i]
        type_loc = os.path.join(data_loc, split, dtype)
        files = os.listdir(type_loc)

        if split == "Train":
          self.train_labels += len(files) * [i]
          self.train_names += files
        elif split == "Val":
          self.val_labels += len(files) * [i]
          self.val_names += files
        elif split == "Test":
          self.test_labels += len(files) * [i]
          self.test_names += files

    # Shuffle data first
    self.N_train = len(self.train_names) - len(self.train_names) % self.batch_size
    self.N_val = len(self.val_names) - len(self.val_names) % self.batch_size
    self.N_test = len(self.test_names) - len(self.test_names) % self.batch_size
    self.shuffle_train()

    self.iter_no = 0
    self.val_no = 0

    self.transform_dict = {
        "Train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        transforms.GaussianBlur(5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
        "Val": transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    }

  # index into file names, labels
  def get_ind(self, index, dset="Train"):
    if dset == "Train":
      fname = self.train_names[index]
      label = self.train_labels[index]
    elif dset == "Val":
      fname = self.val_names[index]
      label = self.val_labels[index]

    label_name = self.label_keys[label]
    img_loc = os.path.join(self.data_loc, dset, label_name, fname)
    input_image = Image.open(img_loc).convert('RGB')
    input_tensor = self.transform_dict[dset](input_image)

    return input_tensor, label

  # For training
  def get_batch(self):
    # End of Epoch
    if self.iter_no + self.batch_size >= self.N_train:
      self.iter_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.iter_no + i)
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.iter_no += self.batch_size
    return batch_img, batch_lab

  def get_val(self):
    if self.val_no + self.batch_size >= self.N_val:
      self.val_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.val_no + i, dset="Val")
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.val_no += self.batch_size
    return batch_img, batch_lab


  def shuffle_train(self):
    inds = np.random.choice(self.N_train, size=self.N_train, replace=False)
    self.train_names = [self.train_names[i] for i in inds]
    self.train_labels = [self.train_labels[i] for i in inds]
    self.iter_no = 0

In [6]:
class BigDataLoader:
  def __init__(self, BatchSize=16, TrainRat=0.9, ValRat=0.05, TestRat=0.05, file_loc="Weather", data_loc="Images/Image", img_size=224, dev="cuda"):
    self.train_names = []
    self.train_labels = []
    self.val_names = []
    self.val_labels = []
    self.test_names = []
    self.test_labels = []

    self.label_keys = {0: "cloudy", 1: "foggy", 2: "rain", 3: "snow", 4: "sunny"}
    self.data_loc = os.path.join(os.getcwd(), data_loc)
    self.file_loc = os.path.join(os.getcwd(), file_loc)
    self.batch_size = BatchSize
    self.img_size = img_size
    self.dev=dev

    # Get file names for weather types
    for w_ind in self.label_keys.keys():
      w_type = self.label_keys[w_ind]
      f = open(os.path.join(file_loc, w_type + ".txt"), 'r')
      lines = [line.strip() for line in f.readlines()]
      f.close()
      # Create a list of keys, files
      N = len(lines)
      self.train_names += lines[:int(N*TrainRat)]
      self.train_labels += [w_ind] * len(lines[:int(N*TrainRat)])

      self.val_names += lines[int(N*TrainRat):int(N*(TrainRat + ValRat))]
      self.val_labels += [w_ind] * len(lines[int(N*TrainRat):int(N*(TrainRat + ValRat))])

      self.test_names += lines[int(N*(TrainRat + ValRat)):]
      self.test_labels += [w_ind] * len(lines[int(N*(TrainRat + ValRat)):])

    # Shuffle data first
    self.N_train = len(self.train_names) - len(self.train_names) % self.batch_size
    self.N_val = len(self.val_names) - len(self.val_names) % self.batch_size
    self.N_test = len(self.test_names) - len(self.test_names) % self.batch_size
    self.shuffle_train()

    self.iter_no = 0
    self.val_no = 0

    self.transform_dict = {
        "Train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        transforms.GaussianBlur(5),
         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
        "Val": transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    }

  # index into file names, labels
  def get_ind(self, index, dset="Train"):
    if dset == "Train":
      fname = self.train_names[index]
      label = self.train_labels[index]
    elif dset == "Val":
      fname = self.val_names[index]
      label = self.val_labels[index]

    label_name = self.label_keys[label]
    img_loc = os.path.join(self.data_loc, label_name, fname)
    input_image = Image.open(img_loc)
    input_tensor = self.transform_dict[dset](input_image)

    return input_tensor, label

  # For training
  def get_batch(self):
    # End of Epoch
    if self.iter_no + self.batch_size >= self.N_train:
      self.iter_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.iter_no + i)
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.iter_no += self.batch_size
    return batch_img, batch_lab

  def get_val(self):
    if self.val_no + self.batch_size >= self.N_val:
      self.val_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.val_no + i, dset="Val")
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.val_no += self.batch_size
    return batch_img, batch_lab


  def shuffle_train(self):
    inds = np.random.choice(self.N_train, size=self.N_train, replace=False)
    self.train_names = [self.train_names[i] for i in inds]
    self.train_labels = [self.train_labels[i] for i in inds]
    self.iter_no = 0

In [7]:
dl = SmallDataLoader()
dl.shuffle_train()
print(dl.N_train, dl.N_val, dl.N_test)

# dl = BigDataLoader()
# dl.shuffle_train()
# print(dl.N_train, dl.N_val, dl.N_test)

2144 256 384


In [8]:
# Example
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
num_ftrs = model.fc.in_features
num_classes = len(dl.label_keys)
model.fc = nn.Linear(num_ftrs, num_classes)
model.to(dl.dev)

batch_im, batch_lab = dl.get_batch()
with torch.no_grad():
    output = model(batch_im)

# Tensor of shape 5
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


tensor([-0.2132,  0.6116, -0.3522,  0.9320,  1.1005], device='cuda:0')


In [9]:
def train_model(model, criterion, optimizer, scheduler, data_loader, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    train_hist = []
    val_hist = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            data_loader.shuffle_train()
            while True:
                if phase == "train":
                  inputs, labels = data_loader.get_batch()
                else:
                  inputs, labels = data_loader.get_val()
                # End of epoch
                if inputs == "EOE":
                  break

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            if phase == "train":
              epoch_loss = running_loss / data_loader.N_train
              epoch_acc = running_corrects / data_loader.N_train
            else:
              epoch_loss = running_loss / data_loader.N_val
              epoch_acc = running_corrects / data_loader.N_val

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == "train":
              train_hist.append(epoch_acc)
            else:
              val_hist.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_hist, val_hist

In [10]:
# Without transfer learning
dl = SmallDataLoader(BatchSize=batch_size)

models = {}
train_hist = {}
val_hist = {}
num_classes = len(dl.label_keys)

for model_name in ["resnet50", "vgg11_bn", "vgg19_bn", "resnet18", "resnet34"]:
  print(model_name)
  model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=False)
  if model_name[:3] == "vgg":
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
  else:
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
  model.to(dl.dev)

  criterion = nn.CrossEntropyLoss()

  # Observe that all parameters are being optimized
  optimizer_ft = optim.Adam(model.parameters(), lr=learning_rate)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

  models[model_name], train_hist[model_name], val_hist[model_name] = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, dl, num_epochs=num_epochs)

resnet50


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 3.2613 Acc: 0.2453
val Loss: 1.5436 Acc: 0.1562
Epoch 1/9
----------
train Loss: 1.5235 Acc: 0.2859
val Loss: 1.5133 Acc: 0.2656
Epoch 2/9
----------
train Loss: 1.5055 Acc: 0.3130
val Loss: 1.5131 Acc: 0.2773
Epoch 3/9
----------
train Loss: 1.4900 Acc: 0.3307
val Loss: 1.5127 Acc: 0.2656
Epoch 4/9
----------
train Loss: 1.4986 Acc: 0.3214
val Loss: 1.5059 Acc: 0.2695
Epoch 5/9
----------
train Loss: 1.5010 Acc: 0.3265
val Loss: 1.5065 Acc: 0.2695
Epoch 6/9
----------
train Loss: 1.4976 Acc: 0.3223
val Loss: 1.5134 Acc: 0.2695
Epoch 7/9
----------
train Loss: 1.4970 Acc: 0.3190
val Loss: 1.5125 Acc: 0.2695
Epoch 8/9
----------
train Loss: 1.4828 Acc: 0.3237
val Loss: 1.5109 Acc: 0.2695
Epoch 9/9
----------
train Loss: 1.4990 Acc: 0.3260
val Loss: 1.5141 Acc: 0.2734
Training complete in 10m 21s
Best val Acc: 0.277344
vgg11_bn


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 2864.5472 Acc: 0.2090
val Loss: 1.6733 Acc: 0.2578
Epoch 1/9
----------
train Loss: 5.6088 Acc: 0.2318
val Loss: 1.5300 Acc: 0.2305
Epoch 2/9
----------
train Loss: 3.0247 Acc: 0.2491
val Loss: 1.5306 Acc: 0.2266
Epoch 3/9
----------
train Loss: 3.0663 Acc: 0.2598
val Loss: 1.5113 Acc: 0.2305
Epoch 4/9
----------
train Loss: 3.5104 Acc: 0.2495
val Loss: 1.5686 Acc: 0.2305
Epoch 5/9
----------
train Loss: 2.9606 Acc: 0.2486
val Loss: 1.5434 Acc: 0.2266
Epoch 6/9
----------
train Loss: 2.9452 Acc: 0.2556
val Loss: 1.5271 Acc: 0.2344
Epoch 7/9
----------
train Loss: 3.3849 Acc: 0.2491
val Loss: 1.5709 Acc: 0.2344
Epoch 8/9
----------
train Loss: 2.8475 Acc: 0.2635
val Loss: 1.5167 Acc: 0.2305
Epoch 9/9
----------
train Loss: 3.4768 Acc: 0.2565
val Loss: 1.5538 Acc: 0.2305
Training complete in 10m 33s
Best val Acc: 0.257812
vgg19_bn


...

Epoch 0/9
----------
train Loss: 2382.0662 Acc: 0.2393
val Loss: 3.2159 Acc: 0.2227
Epoch 1/9
----------
train Loss: 2.3274 Acc: 0.2505
val Loss: 1.5612 Acc: 0.1992
Epoch 2/9
----------
train Loss: 1.6895 Acc: 0.2379
val Loss: 1.5474 Acc: 0.1992
Epoch 3/9
----------
train Loss: 1.7057 Acc: 0.2509
val Loss: 1.5486 Acc: 0.1992
Epoch 4/9
----------
train Loss: 1.6835 Acc: 0.2285
val Loss: 1.5484 Acc: 0.1953
Epoch 5/9
----------
train Loss: 1.7363 Acc: 0.2402
val Loss: 1.5938 Acc: 0.1992
Epoch 6/9
----------
train Loss: 1.6269 Acc: 0.2402
val Loss: 1.7733 Acc: 0.1992
Epoch 7/9
----------
train Loss: 1.6575 Acc: 0.2215
val Loss: 1.5479 Acc: 0.1992
Epoch 8/9
----------
train Loss: 1.7179 Acc: 0.2253
val Loss: 1.5478 Acc: 0.1953
Epoch 9/9
----------
train Loss: 1.7191 Acc: 0.2183
val Loss: 1.8338 Acc: 0.1992
Training complete in 14m 22s
Best val Acc: 0.222656
resnet18
Epoch 0/9
----------


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


train Loss: 2.0096 Acc: 0.2682
val Loss: 1.5021 Acc: 0.3164
Epoch 1/9
----------
train Loss: 1.4300 Acc: 0.3839
val Loss: 1.4343 Acc: 0.3672
Epoch 2/9
----------
train Loss: 1.4147 Acc: 0.3960
val Loss: 1.4292 Acc: 0.3945
Epoch 3/9
----------
train Loss: 1.4337 Acc: 0.3759
val Loss: 1.4298 Acc: 0.3672
Epoch 4/9
----------
train Loss: 1.4168 Acc: 0.4025
val Loss: 1.4243 Acc: 0.3750
Epoch 5/9
----------
train Loss: 1.4147 Acc: 0.3923
val Loss: 1.4297 Acc: 0.3633
Epoch 6/9
----------
train Loss: 1.4231 Acc: 0.3862
val Loss: 1.4307 Acc: 0.3789
Epoch 7/9
----------
train Loss: 1.4287 Acc: 0.3806
val Loss: 1.4248 Acc: 0.3750
Epoch 8/9
----------
train Loss: 1.4115 Acc: 0.3979
val Loss: 1.4391 Acc: 0.3672
Epoch 9/9
----------
train Loss: 1.4106 Acc: 0.3918
val Loss: 1.4275 Acc: 0.3789
Training complete in 8m 14s
Best val Acc: 0.394531
resnet34


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 2.0465 Acc: 0.2523
val Loss: 1.5833 Acc: 0.1289
Epoch 1/9
----------
train Loss: 1.5561 Acc: 0.2304
val Loss: 1.5611 Acc: 0.1914
Epoch 2/9
----------
train Loss: 1.5450 Acc: 0.2528
val Loss: 1.5592 Acc: 0.1914
Epoch 3/9
----------
train Loss: 1.5520 Acc: 0.2449
val Loss: 1.5712 Acc: 0.1836
Epoch 4/9
----------
train Loss: 1.5464 Acc: 0.2528
val Loss: 1.5587 Acc: 0.1914
Epoch 5/9
----------
train Loss: 1.5466 Acc: 0.2575
val Loss: 1.5628 Acc: 0.1875
Epoch 6/9
----------
train Loss: 1.5439 Acc: 0.2612
val Loss: 1.5589 Acc: 0.1914
Epoch 7/9
----------
train Loss: 1.5457 Acc: 0.2561
val Loss: 1.5641 Acc: 0.1875
Epoch 8/9
----------
train Loss: 1.5477 Acc: 0.2523
val Loss: 1.5580 Acc: 0.1914
Epoch 9/9
----------
train Loss: 1.5498 Acc: 0.2500
val Loss: 1.5732 Acc: 0.1836
Training complete in 8m 31s
Best val Acc: 0.191406


In [11]:
print("Not transfer learning")
for model_name in train_hist.keys():
  print(model_name, val_hist[model_name][-1])

Not transfer learning
resnet50 tensor(0.2734, device='cuda:0')
vgg11_bn tensor(0.2305, device='cuda:0')
vgg19_bn tensor(0.1992, device='cuda:0')
resnet18 tensor(0.3789, device='cuda:0')
resnet34 tensor(0.1836, device='cuda:0')


In [12]:
# Using transfer learning
dl = SmallDataLoader(BatchSize=batch_size)

models = {}
train_hist = {}
val_hist = {}
num_classes = len(dl.label_keys)

for model_name in ["resnet50", "vgg11_bn", "vgg19_bn", "resnet18", "resnet34"]:
  print(model_name)
  model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=True)

  # Freeze
  for param in model.parameters():
    param.requires_grad = False

  # Add fc layer
  if model_name[:3] == "vgg":
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
  else:
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
  model.to(dl.dev)

  criterion = nn.CrossEntropyLoss()

  # Observe that all parameters are being optimized
  optimizer_ft = optim.Adam(model.parameters(), lr=learning_rate)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

  models[model_name], train_hist[model_name], val_hist[model_name] = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, dl, num_epochs=num_epochs)

resnet50


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



Epoch 0/9
----------
train Loss: 5.9383 Acc: 0.5070
val Loss: 5.9121 Acc: 0.4531
Epoch 1/9
----------
train Loss: 1.8689 Acc: 0.6679
val Loss: 2.3471 Acc: 0.4883
Epoch 2/9
----------
train Loss: 1.5058 Acc: 0.6870
val Loss: 2.0717 Acc: 0.5234
Epoch 3/9
----------
train Loss: 1.3699 Acc: 0.6996
val Loss: 2.1515 Acc: 0.5078
Epoch 4/9
----------
train Loss: 1.4352 Acc: 0.6814
val Loss: 2.2561 Acc: 0.4961
Epoch 5/9
----------
train Loss: 1.3703 Acc: 0.6908
val Loss: 2.2218 Acc: 0.5078
Epoch 6/9
----------
train Loss: 1.3801 Acc: 0.6964
val Loss: 2.1789 Acc: 0.5000
Epoch 7/9
----------
train Loss: 1.4199 Acc: 0.7029
val Loss: 2.2104 Acc: 0.5039
Epoch 8/9
----------
train Loss: 1.4235 Acc: 0.6940
val Loss: 2.0516 Acc: 0.5234
Epoch 9/9
----------
train Loss: 1.4095 Acc: 0.6880
val Loss: 2.2232 Acc: 0.5078
Training complete in 8m 15s
Best val Acc: 0.523438
vgg11_bn


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to /root/.cache/torch/hub/checkpoints/vgg11_bn-6002323d.pth



Epoch 0/9
----------
train Loss: 7.1361 Acc: 0.4981
val Loss: 7.2309 Acc: 0.4258
Epoch 1/9
----------
train Loss: 5.1484 Acc: 0.6091
val Loss: 6.2024 Acc: 0.4258
Epoch 2/9
----------
train Loss: 4.6965 Acc: 0.6105
val Loss: 6.1832 Acc: 0.4258
Epoch 3/9
----------
train Loss: 4.9283 Acc: 0.5998
val Loss: 6.3405 Acc: 0.4258
Epoch 4/9
----------
train Loss: 5.0451 Acc: 0.6166
val Loss: 6.0985 Acc: 0.4375
Epoch 5/9
----------
train Loss: 4.8717 Acc: 0.6129
val Loss: 6.2737 Acc: 0.4219
Epoch 6/9
----------
train Loss: 4.8074 Acc: 0.6073
val Loss: 6.0016 Acc: 0.4453
Epoch 7/9
----------
train Loss: 4.6648 Acc: 0.6222
val Loss: 6.2014 Acc: 0.4180
Epoch 8/9
----------
train Loss: 4.4322 Acc: 0.6278
val Loss: 6.1356 Acc: 0.4219
Epoch 9/9
----------
train Loss: 4.6522 Acc: 0.6129
val Loss: 6.2172 Acc: 0.4414
Training complete in 7m 54s
Best val Acc: 0.445312
vgg19_bn


...


Epoch 0/9
----------
train Loss: 7.3952 Acc: 0.4949
val Loss: 8.4853 Acc: 0.3203
Epoch 1/9
----------
train Loss: 5.7547 Acc: 0.5756
val Loss: 6.2901 Acc: 0.4023
Epoch 2/9
----------
train Loss: 5.1180 Acc: 0.5951
val Loss: 6.4305 Acc: 0.3867
Epoch 3/9
----------
train Loss: 4.8254 Acc: 0.6063
val Loss: 6.1698 Acc: 0.3828
Epoch 4/9
----------
train Loss: 4.9494 Acc: 0.5993
val Loss: 6.4268 Acc: 0.3984
Epoch 5/9
----------
train Loss: 5.0113 Acc: 0.6012
val Loss: 6.3055 Acc: 0.3984
Epoch 6/9
----------
train Loss: 5.2275 Acc: 0.5812
val Loss: 6.1230 Acc: 0.3984
Epoch 7/9
----------
train Loss: 4.9491 Acc: 0.6049
val Loss: 6.2160 Acc: 0.3828
Epoch 8/9
----------
train Loss: 5.2580 Acc: 0.5826
val Loss: 6.0908 Acc: 0.4102
Epoch 9/9
----------
train Loss: 4.7160 Acc: 0.6096
val Loss: 6.3668 Acc: 0.3828
Training complete in 8m 47s
Best val Acc: 0.410156
resnet18


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 2.9554 Acc: 0.5061
val Loss: 3.3293 Acc: 0.3398
Epoch 1/9
----------
train Loss: 1.2985 Acc: 0.6474
val Loss: 1.6943 Acc: 0.5000
Epoch 2/9
----------
train Loss: 1.1011 Acc: 0.6763
val Loss: 1.6134 Acc: 0.4961
Epoch 3/9
----------
train Loss: 1.0588 Acc: 0.6744
val Loss: 1.6026 Acc: 0.4961
Epoch 4/9
----------
train Loss: 0.9983 Acc: 0.6847
val Loss: 1.6413 Acc: 0.4844
Epoch 5/9
----------
train Loss: 1.0569 Acc: 0.6786
val Loss: 1.7090 Acc: 0.4648
Epoch 6/9
----------
train Loss: 1.0241 Acc: 0.6786
val Loss: 1.6299 Acc: 0.4844
Epoch 7/9
----------
train Loss: 1.0460 Acc: 0.6740
val Loss: 1.6837 Acc: 0.4805
Epoch 8/9
----------
train Loss: 1.0379 Acc: 0.6656
val Loss: 1.6315 Acc: 0.4805
Epoch 9/9
----------
train Loss: 1.0560 Acc: 0.6721
val Loss: 1.6294 Acc: 0.4844
Training complete in 7m 4s
Best val Acc: 0.500000
resnet34


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth



Epoch 0/9
----------
train Loss: 2.7915 Acc: 0.5051
val Loss: 3.0804 Acc: 0.4531
Epoch 1/9
----------
train Loss: 1.4244 Acc: 0.6395
val Loss: 1.8988 Acc: 0.5000
Epoch 2/9
----------
train Loss: 1.0710 Acc: 0.6996
val Loss: 2.0482 Acc: 0.4805
Epoch 3/9
----------
train Loss: 1.1012 Acc: 0.6688
val Loss: 2.0676 Acc: 0.4844
Epoch 4/9
----------
train Loss: 1.1089 Acc: 0.6684
val Loss: 1.9589 Acc: 0.4805
Epoch 5/9
----------
train Loss: 1.0935 Acc: 0.6819
val Loss: 1.9020 Acc: 0.4805
Epoch 6/9
----------
train Loss: 1.0995 Acc: 0.6838
val Loss: 1.9202 Acc: 0.4766
Epoch 7/9
----------
train Loss: 1.1289 Acc: 0.6749
val Loss: 1.9686 Acc: 0.4922
Epoch 8/9
----------
train Loss: 1.1143 Acc: 0.6796
val Loss: 2.0038 Acc: 0.4922
Epoch 9/9
----------
train Loss: 1.1058 Acc: 0.6740
val Loss: 2.0107 Acc: 0.4805
Training complete in 7m 27s
Best val Acc: 0.500000


In [13]:
print("Transfer Learning")
for model_name in train_hist.keys():
  print(model_name, val_hist[model_name][-1])

Transfer Learning
resnet50 tensor(0.5078, device='cuda:0')
vgg11_bn tensor(0.4414, device='cuda:0')
vgg19_bn tensor(0.3828, device='cuda:0')
resnet18 tensor(0.4844, device='cuda:0')
resnet34 tensor(0.4805, device='cuda:0')
